In [21]:
import numpy as np
import pandas as pd

arr = np.array(
    [
        [
            [1, 2, 1],
            [3, 1, 2],
        ],
        [
            [2, 1, 1],
            [1, 0, 3],
        ],
        [
            [3, 2, 1],
            [1, 0, 0],
        ],
    ]
)
print(arr, arr.shape)
print(arr.reshape(-1, 3))
df = pd.DataFrame(arr.reshape(-1, 3), columns=["red", "green", "blue"])
print(df)


[[[1 2 1]
  [3 1 2]]

 [[2 1 1]
  [1 0 3]]

 [[3 2 1]
  [1 0 0]]] (3, 2, 3)
[[1 2 1]
 [3 1 2]
 [2 1 1]
 [1 0 3]
 [3 2 1]
 [1 0 0]]
   red  green  blue
0    1      2     1
1    3      1     2
2    2      1     1
3    1      0     3
4    3      2     1
5    1      0     0
